# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from shutil import copy

from tensorflow.keras.utils import get_file
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Flatten , Dense , Conv2D , MaxPooling2D , Dropout , BatchNormalization , GlobalAveragePooling2D , DepthwiseConv2D

# Downloading the Dataset

In [ ]:
def get_data_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    tf.keras.utils.get_file(
    'food-101.tar.gz',
    'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz',
    cache_subdir='/content',
    extract=True,
    archive_format='tar',
    cache_dir=None
    )
    print("Dataset downloaded and extracted!")

In [ ]:
# Download data and extract it to folder
get_data_extract()

4996292608/4996278331 [==============================] - 257s 0us/step
Dataset downloaded and extracted!


In [ ]:
os.listdir("food-101")

['images', 'license_agreement.txt', 'README.txt', 'meta']

In [ ]:
os.listdir("food-101/meta")

['test.json',
 'labels.txt',
 'test.txt',
 'train.json',
 'classes.txt',
 'train.txt']

In [ ]:
print(f"The No of Classes in Dataset are : {len(os.listdir('/content/food-101/images'))}")

The No of Classes in Dataset are : 101


In [ ]:
count = 0
for folder in os.listdir("food-101/images"):
  folder_path = os.path.join("food-101/images" , folder)
  print(f"The No of Images of Class : {folder} in Dataset are : {len(os.listdir(folder_path))}")
  count = count + 1000

The No of Images of Class : frozen_yogurt in Dataset are : 1000
The No of Images of Class : cheesecake in Dataset are : 1000
The No of Images of Class : risotto in Dataset are : 1000
The No of Images of Class : donuts in Dataset are : 1000
The No of Images of Class : sashimi in Dataset are : 1000
The No of Images of Class : spring_rolls in Dataset are : 1000
The No of Images of Class : panna_cotta in Dataset are : 1000
The No of Images of Class : ravioli in Dataset are : 1000
The No of Images of Class : apple_pie in Dataset are : 1000
The No of Images of Class : pork_chop in Dataset are : 1000
The No of Images of Class : club_sandwich in Dataset are : 1000
The No of Images of Class : peking_duck in Dataset are : 1000
The No of Images of Class : gnocchi in Dataset are : 1000
The No of Images of Class : fried_rice in Dataset are : 1000
The No of Images of Class : chicken_wings in Dataset are : 1000
The No of Images of Class : hot_and_sour_soup in Dataset are : 1000
The No of Images of Cl

#####Dataset Information

This Dataset has 101 Classes with 1000 images for each class



# Splitting the Data into Training and Testing Data

There are two files in meta folder train.txt and test.txt in which list of images is given which needed to be in training set and testing set

* Training Set includes 750 images for each class
* Testing Set incldes 250 images for each class

In [ ]:
def prepare_data(filepath , src_file  , dest_file):
  classes_images = dict()
  for folder_name in os.listdir(src_file):
    classes_images[folder_name] = []
    

  with open(filepath , 'r') as txt:
    #Getting the paths for images
    paths = [read.strip() for read in txt.readlines()]

  for p in paths:
    food = p.split("/")
    classes_images[food[0]].append(food[1] + '.jpg')


  os.makedirs(dest_file , exist_ok = True)
  
  for food in classes_images.keys():
    print("\nCopying Images into " , food)

    if not os.path.exists(os.path.join("food" , food)):
      os.makedirs(os.path.join(dest_file , food))

    for i in classes_images[food]:
  
      copy(os.path.join(src_file , food , i) , os.path.join( dest_file, food , i))
      
    print(f"Copied {len(classes_images[food])} images into {food}")

In [ ]:
# Testing Data
prepare_data("/content/food-101/meta/test.txt" ,"/content/food-101/images" , "food/test")


Copying Images into  frozen_yogurt
Copied 250 images into frozen_yogurt

Copying Images into  cheesecake
Copied 250 images into cheesecake

Copying Images into  risotto
Copied 250 images into risotto

Copying Images into  donuts
Copied 250 images into donuts

Copying Images into  sashimi
Copied 250 images into sashimi

Copying Images into  spring_rolls
Copied 250 images into spring_rolls

Copying Images into  panna_cotta
Copied 250 images into panna_cotta

Copying Images into  ravioli
Copied 250 images into ravioli

Copying Images into  apple_pie
Copied 250 images into apple_pie

Copying Images into  pork_chop
Copied 250 images into pork_chop

Copying Images into  club_sandwich
Copied 250 images into club_sandwich

Copying Images into  peking_duck
Copied 250 images into peking_duck

Copying Images into  gnocchi
Copied 250 images into gnocchi

Copying Images into  fried_rice
Copied 250 images into fried_rice

Copying Images into  chicken_wings
Copied 250 images into chicken_wings

Copy

In [ ]:
# Training Data
prepare_data("/content/food-101/meta/train.txt" , "/content/food-101/images" , "food/train")


Copying Images into  frozen_yogurt
Copied 750 images into frozen_yogurt

Copying Images into  cheesecake
Copied 750 images into cheesecake

Copying Images into  risotto
Copied 750 images into risotto

Copying Images into  donuts
Copied 750 images into donuts

Copying Images into  sashimi
Copied 750 images into sashimi

Copying Images into  spring_rolls
Copied 750 images into spring_rolls

Copying Images into  panna_cotta
Copied 750 images into panna_cotta

Copying Images into  ravioli
Copied 750 images into ravioli

Copying Images into  apple_pie
Copied 750 images into apple_pie

Copying Images into  pork_chop
Copied 750 images into pork_chop

Copying Images into  club_sandwich
Copied 750 images into club_sandwich

Copying Images into  peking_duck
Copied 750 images into peking_duck

Copying Images into  gnocchi
Copied 750 images into gnocchi

Copying Images into  fried_rice
Copied 750 images into fried_rice

Copying Images into  chicken_wings
Copied 750 images into chicken_wings

Copy

## Model Building

In [ ]:
model_path = "/content/drive/MyDrive/Knowledge_distillation/Models/Efficient_Net_Scratch_Teacher_models/EfficientNetScratchmodel.35-0.76.h5"
teacher_model = load_model(model_path)

OSError: ignored

In [ ]:
teacher_model.summary()

In [ ]:
ckpt_path = os.path.join("/content/drive/MyDrive/Knowledge_distillation/Models/Student_Scratch_Model")
os.makedirs(ckpt_path , exist_ok = True)
my_callbacks = [EarlyStopping(patience = 5) ,
                ModelCheckpoint(os.path.join(ckpt_path , "StudentScratchmodel.{epoch:02d}-{val_loss:.2f}.h5"))]

In [ ]:
model = Sequential()
model.add(Conv2D(32  , kernel_size = (3,3), activation = "relu" , input_shape = (380 , 380 , 3)))
model.add(BatchNormalization())
model.add(Conv2D(32  , kernel_size = (3,3),strides = 2 ,  activation = "relu" ))
model.add(MaxPooling2D((2 ,2)))
model.add(Dropout(0.25))
model.add(DepthwiseConv2D(kernel_size = (3 , 3) , strides =1 , use_bias = False ))


model.add(Conv2D(64 , kernel_size = (3,3) , strides = 2 , activation = "relu"))
model.add(BatchNormalization())
model.add(Conv2D(64  , kernel_size = (3,3), activation = "relu" ))
model.add(MaxPooling2D((2, 2))) 
model.add(Dropout(0.2))

model.add(Conv2D(128 ,  kernel_size = (3,3), activation = "relu"))
model.add(BatchNormalization())
model.add(Conv2D(64  , kernel_size = (3 , 3), activation = "relu" ))
model.add(MaxPooling2D((2 , 2) ))
model.add(Dropout(0.25))
model.add(DepthwiseConv2D(kernel_size = (3 , 3) , strides = 5 , use_bias = False ))
model.add(GlobalAveragePooling2D())
#model.add(Conv2D(1792 , kernel_size = (2 , 2) , strides = 1 , activation = "relu"))
model.add(Dense(1792  , activation = "relu"))

model.add(Dense(101 , activation = "relu"))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 378, 378, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 378, 378, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 188, 188, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 94, 94, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 94, 94, 32)        0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 92, 92, 32)       2

44 * 44 * 128 ==Conv2d==> 6 * 6 * 1792 

In [ ]:
model = load_model("/content/drive/MyDrive/Knowledge_distillation/Models/Student_Scratch_Model/StudentScratchmodel.03-11.34.h5")

In [ ]:
model.compile(optimizer = "adam" , metrics = ["accuracy"] , loss = "categorical_crossentropy")

### Data Augementation

In [ ]:
train_datagen = ImageDataGenerator(
                             rotation_range = 180,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True ,
                             vertical_flip = True,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             validation_split=0.2 ,
                             rescale = 1./255
                             
)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/content/food/train" , 
    target_size = (380 , 380),
    class_mode = "categorical" ,
    batch_size = 64

)
test_generator = test_datagen.flow_from_directory(
    "/content/food/test" , 
    target_size = (380 , 380),
    class_mode = "categorical",
    batch_size = 64
    ,
)

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
history = model.fit(train_generator, 
                    epochs = 100,
                    callbacks  = my_callbacks,
                    validation_data = test_generator)

Epoch 1/100
 690/1184 [================>.............] - ETA: 18:17 - loss: 11.3781 - accuracy: 0.0308

KeyboardInterrupt: ignored